# **Training**

In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 106.2 MB/s eta 0:00:00


In [16]:
!mkdir ./data && mv ./train.json ./data && mv ./valid.json ./data && mkdir checkpoint

In [7]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling ,TextDataset

In [6]:
model = GPT2LMHeadModel.from_pretrained("gpt2-medium")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")

In [17]:
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='./data/train.json',
    block_size=32
)
valid_dataset = TextDataset(
    tokenizer=tokenizer, 
    file_path="./data/valid.json", 
    block_size=32
)

/usr/local/lib/python3.8/dist-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [19]:
training_args = TrainingArguments(
    output_dir="./checkpoint",
    overwrite_output_dir=True,
    num_train_epochs=30,
    save_strategy="epoch",
    logging_strategy ="epoch",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=3e-4,
    save_total_limit=2
)

collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                           mlm=False)
trainer = Trainer(model=model.to("cuda"),
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=collator,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4798
  Num Epochs = 30
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 9000
  Number of trainable parameters = 354823168


Epoch,Training Loss,Validation Loss
1,3.585000,4.004187
2,2.431100,4.197192
3,1.434500,4.889631


***** Running Evaluation *****
  Num examples = 450
  Batch size = 16
Saving model checkpoint to ./checkpoint/checkpoint-300
Configuration saved in ./checkpoint/checkpoint-300/config.json
Configuration saved in ./checkpoint/checkpoint-300/generation_config.json
Model weights saved in ./checkpoint/checkpoint-300/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 450
  Batch size = 16
Saving model checkpoint to ./checkpoint/checkpoint-600
Configuration saved in ./checkpoint/checkpoint-600/config.json
Configuration saved in ./checkpoint/checkpoint-600/generation_config.json
Model weights saved in ./checkpoint/checkpoint-600/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 450
  Batch size = 16
Saving model checkpoint to ./checkpoint/checkpoint-900
Configuration saved in ./checkpoint/checkpoint-900/config.json
Configuration saved in ./checkpoint/checkpoint-900/generation_config.json
Model weights saved in ./checkpoint/checkpoint-900/pytorch_model.bin
Deletin

In [ ]:
!mkdir ./model_weight && mkdir ./tokenizer_weight

In [ ]:
model.save_pretrained("./model_weight")
tokenizer.save_pretrained("./tokenizer_weight")

In [ ]:
!mv ./model_weight ./drive/MyDrive/ && mv ./tokenizer_weight ./drive/MyDrive 

# **Inference**

In [ ]:
from transformers import pipeline


GENERATOR = pipeline('text-generation', model='./model/', tokenizer="./tokenizer")

In [ ]:
output = GENERATOR(txt, max_length=50, num_return_sequences=3, num_beams=3, no_repeat_ngram_size=2, early_stopping=True)